In [51]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [9]:
data = pd.read_json("../data/dump.json")

In [14]:
# label = is_bot
# features
# * action
# * change_size <- return to later
# * is_minor
# * is_new
# * is_unpatrolled
# * ns

data.head()

,action,change_size,flags,geo_ip,hashtags,is_anon,is_bot,is_minor,is_new,is_unpatrolled,mentions,ns,page_title,parent_rev_id,rev_id,summary,timestamp,url,user
0,edit,377.0,None,NaN,[],False,False,False,False,False,[],Talk,Talk:Bajadasaurus,883357329.0,883356516.0,/* Images */,2019-02-14 22:43:57.810307,https://en.wikipedia.org/w/index.php?diff=8833...,FunkMonk
1,edit,-66.0,None,NaN,[],False,False,False,False,False,[],Main,The King of Fighters: Maximum Impact,883357327.0,883220913.0,None,2019-02-14 22:43:58.168588,https://en.wikipedia.org/w/index.php?diff=8833...,Benjaminkirsc
2,edit,-4635.0,B,NaN,[],False,True,False,False,False,[],User,User:AlexNewArtBot/WomensHistorySearchResult,883357322.0,883032645.0,"most recent results, added 113, removed 145, t...",2019-02-14 22:43:58.324169,https://en.wikipedia.org/w/index.php?diff=8833...,InceptionBot
3,edit,1.0,None,NaN,[],False,False,False,False,False,[],Template,Template:Samoa-party-stub,883357331.0,398617982.0,None,2019-02-14 22:43:58.613009,https://en.wikipedia.org/w/index.php?diff=8833...,HapHaxion
4,edit,37.0,None,NaN,[],False,False,False,False,False,[],Main,Persela Lamongan,883357330.0,883030131.0,None,2019-02-14 22:43:58.668958,https://en.wikipedia.org/w/index.php?diff=8833...,Afria Nuswantara


In [15]:
# remove entries by anonymous users as these will not be bots
data = data[data.is_anon == False]
data.shape

(6636, 19)

In [33]:
data_subset = data[["action", "is_minor", "is_new", "is_unpatrolled", "ns", "is_bot"]]

In [34]:
data_subset.head()

,action,is_minor,is_new,is_unpatrolled,ns,is_bot
0,edit,False,False,False,Talk,False
1,edit,False,False,False,Main,False
2,edit,False,False,False,User,True
3,edit,False,False,False,Template,False
4,edit,False,False,False,Main,False


In [41]:
# encode Boolean values
data_subset.is_minor.replace([True, False], [1, 0], inplace=True)
data_subset.is_new.replace([True, False], [1, 0], inplace=True)
data_subset.is_bot.replace([True, False], [1, 0], inplace=True)
data_subset.is_unpatrolled.replace([True, False], [1, 0], inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [45]:
# encode categorical values
data_subset = pd.get_dummies(data_subset)

In [48]:
X = data_subset.loc[:, data_subset.columns != "is_bot"]
y = data_subset[["is_bot"]]

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [54]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [55]:
clf.score(X_test, y_test)

0.8876712328767123

In [56]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(X_train,y_train)
clf.score(X_test, y_test)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8899543378995434

In [58]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = clf.predict(X_test)

confusion_matrix(y_test, y_pred)

array([[1839,   31],
       [ 210,  110]])